# User-Specific StackExhange Subdomain Recommender System
LT2: Marvin V. Belina, Kris Gerald R. del Norte, Ray Franco G. Rivera, Ren Christian M. Santos

---------------------------


# Importing Dependencies

Prior to everything, let's import the dependencies of our notebook. At the start of an ElasticMapReduce (EMR) instance with PySpark kernel, we initialize the SparkSession and SparkContext by running the following code.

In [1]:
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1577521624822_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sc

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-0>

By default, the kernel lacks some of the libraries required for this project. We install the following libraries using the `sc.install_pypi_package` command.

In [3]:
sc.install_pypi_package('s3fs')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
sc.install_pypi_package('unicodecsv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Created wheel for unicodecsv: filename=unicodecsv-0.14.1-cp36-none-any.whl size=10767 sha256=e752a1b26934ae35a1c9e86df943eabdd583ef909755317694ceb458da199c4d
  Stored in directory: /var/lib/livy/.cache/pip/wheels/a6/09/e9/e800279c98a0a8c94543f3de6c8a562f60e51363ed26e71283
Successfully built unicodecsv

In [5]:
sc.install_pypi_package('pandas')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

After we have downloaded and installed all the dependencies, we now import them below.

In [6]:
# Importing libraries
import xml.etree.ElementTree as ET
import s3fs
import re
import os, glob
import string
import unicodecsv as csv
import pandas as pd

from pyspark.ml.recommendation import ALS
from pyspark.ml.feature import StringIndexer, QuantileDiscretizer, Bucketizer
from pyspark.sql.functions import min, max, col, collect_set, size
from pyspark.sql.window import Window
from pyspark.ml.fpm import FPGrowth
from pyspark.ml.evaluation import RegressionEvaluator

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

---------------------

# Executive Summary

StackExchange is a Q&A forum that empowers users to tap knowledge through an easily accessible platform. However, given the limited functionality of their website and the vast amount of subdomains available, it hampers the growth of knowledge sharing and the website by making it difficult for users to discover new content that would be interesting for them. With the culture that StackExchange promotes, and considering the potential for a specific user to learn and grow, it would be beneficial to create a recommendation of topics tailored to what the user likes, to help them further expand their knowledge base. In this project, the team wrangled ~60GB worth of user-contributed content gathered across 162 subdomains from the website, StackExchange. This was then used to create two (2) implementations of a recommender system using Frequent Itemset Mining (FIM) & Association Rules (AR) and User-based Collaborative Filtering through the cloud computing service, Amazon Web Services (AWS). The team was able to create two (2) recommender systems that are beneficial for two (2) endpoints -- website owners and users. Website owners could use the recommender system to generate more web traffic for their specific subdomains and users can easily discover new content through the user-specific recommender system that we have created. The approach that the team has created can also be applied to various industries such as e-commerce, social media and any other industry that has a rich database of user-item transactions.

# Introduction

StackExchange is an aggregator website of various subdomains that operate similar to a question-and-answer (Q&A) forum. The website is modelled after the initial site StackOverflow, primarily focused on computer programming questions. With StackExchange, the scope of topics, questions and discussions are vaster, making it harder to discover new content that is most relevant to a user’s interest. Even though the home page shows the top questions based on “hotness”, these recommendations are not user-specific. This led the team to ask the question -- "How do we make it easier for users of StackExchange to discover new content that would suit their interests?" Currently, there’s no built-in system to recommend new subdomains that are specific to a user. Being able to create a user-specific recommender system would be beneficial for the owner (e.g. more website traffic) and users (e.g. content discovery) of the website.

In this project, the team proposes to create two (2) implementations of a recommender system using:
1. Frequent Itemset Mining; and
2. User-based Collaborative Filtering

For the implementation of FIM, we will process the gathered data to create an itemset of visited subdomains for each specific user. Found frequent item-sets indicate good combinations of subdomains that are frequented together by users. In addition, we can also establish associations between these subdomains to further our analysis.

For the implementation of a User-based Collaborative Filtering, we will be creating a utility matrix with the users as rows and subdomains as the columns. The ratings of each user for a certain subdomain will be based on how much the user frequents the subdomain. We plan to measure this through comment counts. With this information, we can recommend new subdomains to a certain user based on the most similar users. 

# Data Collection and Description

The data used in this project was collected from the StackExchange dataset available at https://archive.org/details/stackexchange. The dataset contains ~60GB worth of user-contributed content across 162 subdomains. Each subdomain dump contains seven (7) XML files – badges.xml, comments.xml, posts.xml, posthistory.xml, postlinks.xml users.xml, and votes.xml.

The full schema of each .xml file can be found at https://ia800107.us.archive.org/27/items/stackexchange/readme.txt.

For this particular project, we would only be needing data from the comments.xml file of each subdomain. In total, we will be using 16.5 million user comments from about 248,000 unique users across 162 subdomains to create our recommender system.

Since we are dealing with xml files, we create a function that will take as an input the subdomain name and read its contents as a string. After which, we then parse the string as an xml file.

In [7]:
# Parse xml
def parsexml(fp):
    f = sc.textFile('s3://bdcc-rrivera-2020/stackexchange/actual/' + fp + '.stackexchange.com/Comments.xml').collect()
    root = ET.fromstring(' '.join(f))
    return root

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Now, let's get the list of all subdomain names from `s3://bdcc-rrivera-2020/stackexchange/actual/`, which is the path where all the stackexchange subdomains archive resides.

In [8]:
# Getting list of subdomain names
# key = 'mysecretkey'
# secret = 'mysecretid'
# fs = s3fs.S3FileSystem(key=key, secret=secret)
# fps3 = fs.ls('s3://bdcc-rrivera-2020/stackexchange/actual/')

# filepaths = []

# for e in range(len(fps3)):
#     try:
#         m = re.findall('actual/(.+).stackexchange.com', fps3[e])[0]
#         filepaths.append(m)
#     except:
#         continue

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Using the `parsexml` function we created earlier, we parse each row (comment) of a subdomain and save it as a `csv` file. This will allow us to easily read our big data though PySpark.

In [9]:
# Merge all comments per subdomain and save to csv

# cols = ['Id', 'PostId', 'Score', 'Text', 'CreationDate', 'UserId', 'SubDomain']

# for fp in filepaths:
#     commentlist = []
#     root = parsexml(fp)
#     for i in range(len(root)):
#         root[i].attrib['SubDomain'] = fp # Add subdomain name as key-value pair
#         if 'UserDisplayName' in root[i].attrib.keys():
#             del root[i].attrib['UserDisplayName']
#         if cols == list(root[i].attrib.keys()):
#             del root[i].attrib['Text']
#             commentlist.append(root[i].attrib)
#         else:
#             continue

#     keys = commentlist[0].keys()
#     with fs.open('s3://bdcc-rrivera-2020/stackexchange/actual/csv/' + fp + '.csv', 'wb') as output_file:
#         dict_writer = csv.DictWriter(output_file, keys)
#         dict_writer.writeheader()
#         dict_writer.writerows(commentlist)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Spark can read the csv files that we have created as one big file using the function `spark.read.csv`. The function will return a spark dataframe.

In [10]:
# Reading all csv files
df = spark.read.csv('s3://bdcc-rrivera-2020/stackexchange/actual/csv/*.csv', 
                    header=True,
                    schema="""
                    Id FLOAT,
                    PostId FLOAT,
                    Score FLOAT,
                    CreationDate STRING,
                    UserId FLOAT,
                    SubDomain STRING
                    """).cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

After reading the files, let's look at how many rows we have for the whole dataframe.

In [11]:
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

16530530

We have a total count of rows (comments) of 16,530,530.

In [12]:
df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+------+-----+--------------------+------+-----------+
| Id|PostId|Score|        CreationDate|UserId|  SubDomain|
+---+------+-----+--------------------+------+-----------+
|1.0|  19.0|  0.0|2009-10-30T20:54:...|  24.0|electronics|
|2.0|  19.0|  0.0|2009-10-30T21:18:...|  12.0|electronics|
|3.0|  46.0|  0.0|2009-10-31T00:43:...|  24.0|electronics|
|4.0|  46.0|  2.0|2009-11-01T02:25:...|  26.0|electronics|
|5.0|  46.0|  1.0|2009-11-01T03:32:...|  26.0|electronics|
+---+------+-----+--------------------+------+-----------+
only showing top 5 rows

Printing out the first five (5) rows of our dataframe, we can see that it has five (5) columns namely Id, PostId, Score, CreationDate, UserId, and Subdomain. Let's drop NaNs and check the column types.

In [13]:
# Dropping NaNs
# df = df.withColumn('Id', df['Id'].astype('float'))
# df = df.withColumn('PostId', df['PostId'].astype('float'))
# df = df.withColumn('Score', df['Score'].astype('float'))
# df = df.withColumn('UserId', df['UserId'].astype('float'))
df = df.dropna()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
# Checking column types
df.dtypes

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('Id', 'float'), ('PostId', 'float'), ('Score', 'float'), ('CreationDate', 'string'), ('UserId', 'float'), ('SubDomain', 'string')]

After dropping NaNs and checking for our column data types, let's recount the number of rows to see if we dropped some rows.

In [15]:
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

16530530

Since the number of rows is still 16,530,530, there are no NaN values in our dataframe.

After preprocessing, we now have our working dataframe shown below.

In [16]:
df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+------+-----+--------------------+------+-----------+
| Id|PostId|Score|        CreationDate|UserId|  SubDomain|
+---+------+-----+--------------------+------+-----------+
|1.0|  19.0|  0.0|2009-10-30T20:54:...|  24.0|electronics|
|2.0|  19.0|  0.0|2009-10-30T21:18:...|  12.0|electronics|
|3.0|  46.0|  0.0|2009-10-31T00:43:...|  24.0|electronics|
|4.0|  46.0|  2.0|2009-11-01T02:25:...|  26.0|electronics|
|5.0|  46.0|  1.0|2009-11-01T03:32:...|  26.0|electronics|
+---+------+-----+--------------------+------+-----------+
only showing top 5 rows

# Exploratory Data Analysis

The Exploratory Data Analysis (EDA) stage will allow us to uncover the main characteristics of the dataset. Let us create a temporary SQL table to easily query the information that we want to retrieve.

In [17]:
# Temporary SQL table
df.createOrReplaceTempView('df')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
# Unique users
spark.sql('''
SELECT count(distinct(UserId)) as Unique_Users
FROM df
''').toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   Unique_Users
0        247964

Retrieving the total number of unique users, we see that there are about 248,000 users in StackExchange!

In [19]:
# Unique subdomains
spark.sql('''
SELECT count(distinct(Subdomain)) as Unique_Subdomain
FROM df
''').toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   Unique_Subdomain
0               162

Retrieving the total number of unique subdomains, we see that there are 162 subdomains in StackExchange!

Now, let's take a look at the distribution of comments per user across various subdomains.

In [20]:
# Engagement of users across subdomains
spark.sql('''
SELECT UserId, count(distinct(SubDomain)) as Presence_in_Subdomain
FROM df
GROUP BY UserId
ORDER BY Presence_in_Subdomain DESC
LIMIT 10
''').toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   UserId  Presence_in_Subdomain
0    92.0                     98
1    13.0                     98
2    29.0                     96
3    43.0                     95
4    38.0                     93
5    44.0                     92
6    36.0                     90
7    69.0                     89
8    10.0                     89
9    20.0                     89

Looking at the top ten (10) users, we can see that they have commented across more than 89 subdomains, which is about half of the total subdomains in StackExchange. This means that these users have engaged in a wide array of subdomains. However, this does not mean that they are active users of each subdomain.

In [21]:
# # Save csv to s3 bucket
# spark.sql('''
# SELECT UserId, count(distinct(SubDomain)) as Presence_in_Subdomain
# FROM df
# GROUP BY UserId
# ORDER BY Presence_in_Subdomain DESC
# ''').toPandas().to_csv('s3://bdcc-rrivera-2020/stackexchange/eda/4.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Let's take a look at the count of unique users and comments of each subdomain to see which is most popular / active among users.

In [22]:
# Count of unique users and total comments per subdomain
spark.sql('''
SELECT SubDomain, count(distinct(UserId)) as Unique_Users,
count(Id) as Total_Comments
FROM df
GROUP BY SubDomain
ORDER BY Unique_Users DESC
''').toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

       SubDomain  Unique_Users  Total_Comments
0           unix         64806          675109
1            tex         46862          877205
2          stats         45311          526827
3    electronics         42947          786504
4          apple         42621          319778
..           ...           ...             ...
157    esperanto           276            4300
158    ukrainian           271            5721
159           or           216            1989
160        tezos           192            1448
161      conlang           171            1259

[162 rows x 3 columns]

The most popular subdomain among users with 64,806 unique users is unix.stackexchange.com which is a subdomain for anything related to users of Linux, FreeBSD and other Unix-like operating systems. In contrast, the most commented subdomain is tex.stackexchange.com which is a subdomain for users of TeX, LaTeX, ConTeXt, and related typesetting systems.

We can see that StackExchange is still mainly focused on technical people and is having trouble gaining users from non-technical subdomains such as conlang.stackexchange.com and ukrainian.stackexchange.com. This finding reinforces the value of this project by allowing users to discover new content that would suit their interests.

In [23]:
# # Save csv to s3 bucket
# spark.sql('''
# SELECT SubDomain, count(distinct(UserId)) as Unique_Users,
# count(Id) as Total_Comments
# FROM df
# GROUP BY SubDomain
# ORDER BY Unique_Users DESC
# ''').toPandas().to_csv('s3://bdcc-rrivera-2020/stackexchange/eda/5.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Methods and Results

This section will discuss the methods used in developing the recommender systems as well as the corresponding results and insights about the system.

## User-based Collaborative Filtering Recommender System

In creating a user-based collaborative filtering recommender system, our goal is to recommend suitable subdomains to a specific user. As mentioned in the introduction, this information would be very valuable for users in terms of content discovery as well as the owners of website, to generate more website traffic. PySpark provides a wonderful implementation of recommender systems using Alternating Least Squares (ALS). For this project, we will be using this library.

The methodology in creating a user-based collaborative filtering recommender system is given below:
1. Create numeriacl subdomain categories;
2. Create subdomain mapping dictionary;
3. Create uid, itemid, rating dataframe;
4. Convert columns to float;
5. Bin the ratings column;
6. Split the dataset into train/test (80/20);
7. Implement ALS.

First, we pass the SubDomain column to the `StringIndexer` function to perform a one-hot encode of the column. This allows the computer to treat the variables as numerical.

In [24]:
# Creating numerical subdomain categories
indexer = StringIndexer(inputCol="SubDomain", outputCol="SubDomainIndex")
df_indexed = indexer.fit(df)
df = df_indexed.transform(df)
df = df.withColumn('SubDomainIndex', df['SubDomainIndex'].astype('float'))
df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+------+-----+--------------------+------+-----------+--------------+
| Id|PostId|Score|        CreationDate|UserId|  SubDomain|SubDomainIndex|
+---+------+-----+--------------------+------+-----------+--------------+
|1.0|  19.0|  0.0|2009-10-30T20:54:...|  24.0|electronics|           1.0|
|2.0|  19.0|  0.0|2009-10-30T21:18:...|  12.0|electronics|           1.0|
|3.0|  46.0|  0.0|2009-10-31T00:43:...|  24.0|electronics|           1.0|
|4.0|  46.0|  2.0|2009-11-01T02:25:...|  26.0|electronics|           1.0|
|5.0|  46.0|  1.0|2009-11-01T03:32:...|  26.0|electronics|           1.0|
+---+------+-----+--------------------+------+-----------+--------------+
only showing top 5 rows

Now, let's creating a mapping dictionary so we can later retrieve each subdomain after our system has recommended to a certain user.

In [25]:
# Creating subdomain mapping dictionary
subdomain_dict = {}

for k, v in enumerate(df_indexed.labels):
    subdomain_dict[k] = v

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

The following dictionary provides a mapping of all the subdomains to its specific index. The keys are the subdomain index while the values are its actual subdomain name.

In [26]:
subdomain_dict

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{0: 'tex', 1: 'electronics', 2: 'english', 3: 'physics', 4: 'unix', 5: 'stats', 6: 'softwareengineering', 7: 'scifi', 8: 'worldbuilding', 9: 'gis', 10: 'mathematica', 11: 'apple', 12: 'wordpress', 13: 'gaming', 14: 'magento', 15: 'codegolf', 16: 'salesforce', 17: 'ell', 18: 'security', 19: 'codereview', 20: 'dba', 21: 'workplace', 22: 'academia', 23: 'travel', 24: 'drupal', 25: 'judaism', 26: 'diy', 27: 'sharepoint', 28: 'gamedev', 29: 'rpg', 30: 'puzzling', 31: 'blender', 32: 'ux', 33: 'android', 34: 'money', 35: 'photo', 36: 'cs', 37: 'aviation', 38: 'graphicdesign', 39: 'chemistry', 40: 'philosophy', 41: 'cooking', 42: 'raspberrypi', 43: 'bicycles', 44: 'webmasters', 45: 'skeptics', 46: 'politics', 47: 'history', 48: 'music', 49: 'crypto', 50: 'arduino', 51: 'german', 52: 'mechanics', 53: 'biology', 54: 'christianity', 55: 'movies', 56: 'space', 57: 'dsp', 58: 'japanese', 59: 'ethereum', 60: 'bitcoin', 61: 'hinduism', 62: 'emacs', 63: 'cstheory', 64: 'law', 65: 'writers', 66: 'webap

The PySpark ALS function only allows as input a spark DataFrame with the following schema -- uid, itemid, rating (count). As such, we create a spark DataFrame to be able to develop our recommender system. This is equivalent to creating our utility matrix.

In [27]:
# Creating uid, itemid, rating (count) dataframe
df_rs = spark.createDataFrame(df.groupby(['UserId', 'SubDomainIndex']).count().collect())
df_rs.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------+-----+
| UserId|SubDomainIndex|count|
+-------+--------------+-----+
|   77.0|           2.0| 1075|
| 8843.0|           2.0|    2|
| 8293.0|           2.0|    2|
|10117.0|           2.0|    1|
|11007.0|           2.0|  146|
+-------+--------------+-----+
only showing top 5 rows

If we want to remove users with only one (1) subdomain visited or commented on, we can run the code below. The reason for this is that users who have commented only once in a subdomain may just be added noise during the training of our recommender system. However, for the final recommender system, we left it as such since it generates a better evaluation metric score.

In [28]:
# RUN IF YOU WANT TO REMOVE ALL USERS WITH ONLY 1 SUBDOMAIN VISITED!

# # Selecting users with only 1 subdomain visited
# df_rs2 = (df_rs.groupby('UserId')
#           .agg(collect_set('SubDomainIndex').alias('trans'))
#           .where(size(col('trans')) == 1)
#           .select('UserId'))

# # Removing users with only 1 subdomain visited
# df_rs3 = df_rs.join(
#     df_rs2, 
#     [df_rs.UserId == df_rs2.UserId], 
#     how='left_anti'
# )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

As mentioned earlier, the PySpark ALS function only takes as an input a spark DataFrame. In addition, it requires that all fields should be of float type. As such, we convert all fields to float.

In [29]:
# Converting UserId, SubDomainIndex to float
df_rs = df_rs.withColumn('UserId', df_rs['UserId'].astype('float'))
df_rs = df_rs.withColumn('SubDomainIndex', df_rs['SubDomainIndex'].astype('float'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

We also have the option of scaling the rating (count) column if we run the code below. However, similar with removing users with only one (1) subdomain visited, we chose to leave it as such for the final recommender system since it produces a better recommender system performance.

In [30]:
# Scaling counts per subdomain
# w = Window.partitionBy("SubDomainIndex")

# scaled_result = (col("count") - min("count").over(w)) / (max("count").over(w) - min("count").over(w))

# df_rs = df_rs.withColumn("count_scaled", scaled_result)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Now, having the raw comment counts as ratings may affect the accuracy of our recommender system due to its continuous nature. In order to solve this, we decided to arbitrarily bin the ratings (count) column into five discrete categories. Comment counts between 1-10 will have an engagement score of 1.0, 10-100 will be equal to 2.0, 100-500 will be 3.0, 500-1000 is 4.0, and 1000 and above will have a perfect rating of 5.0. This binning will hopefully capture the engagement of each user for each subdomain better.

In [31]:
# Binning (franco-defined)
bucketizer = Bucketizer(splits=[1, 10, 100, 500, 1000, float("inf")],
                        inputCol="count", outputCol="rating")
bucketed = bucketizer.setHandleInvalid("keep").transform(df_rs)
bucketed = bucketed.withColumn('rating', bucketed['rating'] + 1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Let's take a look at the distinct ratings we created below.

In [32]:
# Distinct ratings
bucketed.select('rating').distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+
|rating|
+------+
|   1.0|
|   4.0|
|   3.0|
|   2.0|
|   5.0|
+------+

In [33]:
df_rs = bucketed.drop('count')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

To train our recommender system, we split the data into two (2). The splitting is done with a ratio of 80/20 for train test and test set respectively. The ALS algorithm attempts to estimate the ratings matrix R as the product of two lower-rank matrices, X and Y, i.e. X * Yt = R.

In [34]:
# Train/test split
df_training, df_test = df_rs.randomSplit([0.8, 0.2], seed=42)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Now, we can fit our ALS algorithm to the training set. The function takes the following parameters:
- **maxIter** - number of ALS iterations to use
- **regParam** - regularization parameter
- **userCol** - user column
- **itemCol** - item column
- **ratingCol** - rating column
- **coldStartStrategy** - strategy for dealing with unknown or new users/items at prediction time

In [35]:
# ALS
als = ALS(maxIter=5, regParam=0.01, userCol="UserId", itemCol="SubDomainIndex", 
          ratingCol="rating", coldStartStrategy="drop")

als_trained = als.fit(df_training)
df_predict = als_trained.transform(df_test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

We then save the results by running the following code below.

In [36]:
# df_predict.toPandas().to_csv('s3://bdcc-rrivera-2020/stackexchange/eda/testsetpredictions.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

We can evaluate our recommender system by testing it on our test set. With this, we obtained a Root-mean-square error (RMSE) of 0.64. This value is relative to the set rating of 1 to 5 which means that the results are fairly acceptable.

In [37]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(df_predict)
print("Root-mean-square error (RMSE) = " + str(rmse))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Root-mean-square error (RMSE) = 0.631530670539948

With our recommender system created, let's try to make ten (10) subdomain recommendations to 20 users of StackExchange.

In [38]:
# Make recommendation
users = df_rs.select(als.getUserCol()).distinct().limit(20) # Limit to 20 users only
df_recomm = als_trained.recommendForUserSubset(users, 5) # Recommend 10 subdomains to 20 users

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Looking at the dataframe below, we can see that our recommender system was able to recommend subdomains for 20 users. The dataframe shows the recommendations as a tuple of (subdomain index, rating). However, a dataframe form is not the best way to visualize the recommendations offered to us by the system as it cuts off the succeeding recommendations.

In [39]:
df_recomm.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------------------+
|UserId|     recommendations|
+------+--------------------+
|151313|[[54, 1.4390156],...|
| 46216|[[80, 2.7420862],...|
|158615|[[58, 1.3765002],...|
|199107|[[89, 1.4872445],...|
| 40068|[[29, 3.4181402],...|
|174591|[[61, 3.1779935],...|
| 53788|[[15, 1.0364293],...|
| 87436|[[89, 2.1108139],...|
| 49045|[[2, 1.0462611], ...|
| 94226|[[58, 1.1530683],...|
|106426|[[58, 2.440846], ...|
|234576|[[4, 0.9703308], ...|
|251851|[[89, 1.1534979],...|
| 73847|[[143, 1.072845],...|
|111682|[[54, 1.2653142],...|
| 42656|[[14, 2.642518], ...|
|  2976|[[51, 2.00029], [...|
|174463|[[58, 1.259797], ...|
|166559|[[77, 1.1249527],...|
|  4618|[[16, 2.9131286],...|
+------+--------------------+

In [40]:
# df_recomm.toPandas().to_csv('s3://bdcc-rrivera-2020/stackexchange/eda/samplerecomm.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Instead, we can try to single out one (1) user and print out the recommended subdomains using the subdomain mapping dictionary we created earlier. Let's try to recommend five (5) subdomains for user 40124.

In [41]:
# Getting user 40124.0
users2 = df_rs.filter(df_rs['UserId'] == 40124.0 ).distinct().select('UserId').limit(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
# Creating recommender for prof alis
df_recomm2 = als_trained.recommendForUserSubset(users2, 5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In the array below, we can see the recommended subdomain indices and its corresponding rating for user 40124. The recommendations are arranged by decreasing rating already.

In [43]:
# Getting recommended items
df_recomm2.toPandas()['recommendations'].values

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

array([list([Row(SubDomainIndex=29, rating=2.0890419483184814), Row(SubDomainIndex=69, rating=2.0499415397644043), Row(SubDomainIndex=75, rating=1.9870492219924927), Row(SubDomainIndex=98, rating=1.9799880981445312), Row(SubDomainIndex=37, rating=1.9656609296798706)])],
      dtype=object)

Let's try to identify which subdomains were recommended to us by accessing the look-up dictionary we created earlier.

In [45]:
# Look-up recommended ids
print(subdomain_dict[29])
print(subdomain_dict[69])
print(subdomain_dict[75])
print(subdomain_dict[98])
print(subdomain_dict[37])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rpg
linguistics
parenting
chess
aviation

We can see that our system has recommended specific subdomains that our user may be interested in based on the interests of other users that are similar to him.

## Frequent Itemset Mining

Even though we were able to create a user-specific recommender system for StackExchange, we would also like to identify frequent itemsets of subdomains, or subdomains commonly visited together by many users. It is also in our interest to create association rules between them to suggest possible subdomain collaborations. For this part of the project, we will be using FP-growth algorithm to implement FIM for the reason that it does not require the generation of a candidate itemset. Instead, it uses projected databases by scanning the subset of the database that contains the prefix of the itemsets and with the prefix removed from the values. This generally translates to a faster run-time.

The methodology to identify frequent itemsets and create association rules of artists is given below:
1. Create horizontal transactional database;
2. Create frequent itemsets via FP-growth;
3. Create association rules (confidence, lift)

Given below is the original dataframe that we have from the comments.xml file.

In [46]:
df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+------+-----+--------------------+------+-----------+--------------+
| Id|PostId|Score|        CreationDate|UserId|  SubDomain|SubDomainIndex|
+---+------+-----+--------------------+------+-----------+--------------+
|1.0|  19.0|  0.0|2009-10-30T20:54:...|  24.0|electronics|           1.0|
|2.0|  19.0|  0.0|2009-10-30T21:18:...|  12.0|electronics|           1.0|
|3.0|  46.0|  0.0|2009-10-31T00:43:...|  24.0|electronics|           1.0|
|4.0|  46.0|  2.0|2009-11-01T02:25:...|  26.0|electronics|           1.0|
|5.0|  46.0|  1.0|2009-11-01T03:32:...|  26.0|electronics|           1.0|
+---+------+-----+--------------------+------+-----------+--------------+
only showing top 5 rows

Let's create a horizontal database by grouping the dataframe according to UserId and then getting all the unique subdomains that a unique user has visited. In the following cell, we do so and then train the FP-growth algorithm as well to the horizontal database that we just created with the following hyperparameters:

- $minSupport = 0.01$
- $minConfidence = 0.01$

Setting $minSupport = 0.01$ means that we are getting only the itemsets with a relative support equal to $0.01%$. Relative support is defined as the fraction of transactions having $X$ as a subset,
$$
relSup(X) = \frac{sup(X)}{N}.
$$

Setting $minConfidence = 0.01$ sets the minimum confidence for generating an Association Rule.

In [47]:
df_trans = (df.groupby('UserId')
                     .agg(collect_set('SubDomain').alias('trans'))
                     .cache())

fpgrowth = FPGrowth(itemsCol="trans", minSupport=0.01, minConfidence=0.01)
fpgrowth_trained = fpgrowth.fit(df_trans)

# print('Frequent itemsets:')
df_fim = fpgrowth_trained.freqItemsets.orderBy('freq', ascending=False)

# print('Association rules:')
df_ar = fpgrowth_trained.associationRules

# print('Consequents from association rules:')
df_car = fpgrowth_trained.transform(df_trans)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Shown below is the horizontal database that we have created which contains all the unique subdomains each unique user has commented on.

In [48]:
df_trans.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------------------+
|UserId|               trans|
+------+--------------------+
| 714.0|[fitness, network...|
|1033.0|[mathematica, hea...|
|1575.0|[webapps, mechani...|
|2077.0|[ai, networkengin...|
|2098.0|[parenting, mathe...|
+------+--------------------+
only showing top 5 rows

Looking at its count, there are 247,964 unique users in total -- consistent with our finding during the EDA.

In [49]:
df_trans.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

247964

We were able to find a total of 573 frequent itemsets, as seen below.

In [50]:
df_fim.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

573

We can see that most frequent itemsets are only 1-itemsets and 2-itemsets. With this knowledge, we can see which subdomains are commonly visited together by different users. We can then try to encourage interaction between the frequent-itemset subdomains to improve community engagement.

In [51]:
df_fim.toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                        items   freq
0                      [unix]  64806
1                       [tex]  46862
2                     [stats]  45311
3               [electronics]  42947
4                     [apple]  42621
..                        ...    ...
568          [ethereum, unix]   2498
569     [mathematica, travel]   2497
570       [webmasters, scifi]   2492
571            [photo, stats]   2492
572  [webmasters, salesforce]   2483

[573 rows x 2 columns]

In [52]:
# Saving fim results to csv
# df_fim.toPandas().to_csv('s3://bdcc-rrivera-2020/stackexchange/eda/fimresults.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
df_fimpandas = df_fim.toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Now, getting only 2-itemsets, we see a better look at which subdomains are commonly visited. We can see that tex and unix top the chart with 11,156 visits from unique users.

In [54]:
# Getting only fim with more than 2 items
df_fimpandas['len(items)'] = df_fimpandas['items'].apply(lambda x: len(x))
df_fimpandas[df_fimpandas['len(items)'] == 2]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                        items   freq  len(items)
36                [tex, unix]  11156           2
38              [stats, unix]   9966           2
40               [stats, tex]   9460           2
42        [electronics, unix]   9251           2
43         [electronics, tex]   9154           2
..                        ...    ...         ...
568          [ethereum, unix]   2498           2
569     [mathematica, travel]   2497           2
570            [photo, stats]   2492           2
571       [webmasters, scifi]   2492           2
572  [webmasters, salesforce]   2483           2

[482 rows x 3 columns]

This also shows that users mostly visit only 2 subdomains rather than more.

The mentioned hyperparameter values were decided based on the variety of items between itemsets and the number of itemsets returned by the algorithm. We wanted to get enough itemsets which vary from each other quite well. In this case, we got the best result using the mentioned hyperparameters.

The insights that can be gathered from the frequent itemset analysis is limited in the sense that it only provides the set of subdomains which are frequently visited together. It does not provide associations between a subdomain and a set of subdomains. To create this association, we perform association rule mining. The association between subdomains and the sets of subdomains is measured by $confidence$ and $lift$.

$Confidence$ refers to the likelihood that subdomain B is also visited if subdomain A is visited. <br>
$Lift$ refers to the increase in the ratio of visits of B when A is visited. Here are the general rules of thumb for the $Lift$ value:
- $Lift = 1$ means no association between artists A and B. 
- $Lift > 1$ means products A and B are more likely to be associated with each other. 
- $Lift < 1$ means products A and B are unlikely to be associated with each other.

With this knowledge, we find that there are a total of 964 association rules created.

In [55]:
df_ar.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

964

Ordering all of these association rules by decreasing confidence, we see that biology will most likely be visited if a user visits the subdomain tex.

In [57]:
df_ar.orderBy('confidence', ascending=False).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+----------+-------------------+------------------+
|     antecedent|consequent|         confidence|              lift|
+---------------+----------+-------------------+------------------+
|      [biology]|     [tex]| 0.3612705191512078|1.9116145920107996|
|          [dsp]|     [tex]| 0.3605310468814825|1.9077017734821378|
|    [mechanics]|     [tex]| 0.3585034789551728|1.8969731692125917|
|     [puzzling]|     [tex]|0.35273626250140466|1.8664567153535554|
| [softwarerecs]|     [tex]| 0.3459727700725283|1.8306686005348558|
|     [ethereum]|     [tex]|0.34062068099007414|1.8023487375917107|
|[worldbuilding]|     [tex]|0.33345642540620385|1.7644400381849672|
|       [crypto]|     [tex]|0.33200047042220393|1.7567360472829026|
|          [rpg]|     [tex]| 0.3319880073800738|1.7566701007637877|
|      [cooking]|     [tex]|  0.325875260839323|1.7243253207025286|
|    [chemistry]|     [tex]|0.32541069556099267| 1.721867135719474|
|  [mathematica]|     [tex]| 0.3226930186264752|

Ordering the association rules by decreasing lift, we can see that worldbuilding and mathematica are most associated with each other.

In [56]:
df_ar.orderBy('lift', ascending=False).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+---------------+-------------------+------------------+
|     antecedent|     consequent|         confidence|              lift|
+---------------+---------------+-------------------+------------------+
|[worldbuilding]|  [mathematica]|0.22119645494830134|3.8993855932603867|
|  [mathematica]|[worldbuilding]| 0.2129247831650789| 3.899385593260386|
|   [salesforce]|[worldbuilding]|0.20217818020642056| 3.702578306994451|
|[worldbuilding]|   [salesforce]|0.23581979320531757|3.7025783069944507|
|   [salesforce]|  [mathematica]|0.20015196606091307|3.5284005482957665|
|  [mathematica]|   [salesforce]|  0.224726290345514|3.5284005482957657|
|       [drupal]|[worldbuilding]|  0.182606393540274| 3.344151533812445|
|[worldbuilding]|       [drupal]| 0.2037666174298375|3.3441515338124446|
|      [blender]|  [mathematica]| 0.1843353955192504| 3.249576426456378|
|  [mathematica]|      [blender]|0.21818569600454998| 3.249576426456378|
|       [drupal]|  [mathematica]|0.1841286650340856

In [58]:
# Saving ar to csv
# df_ar.toPandas().to_csv('s3://bdcc-rrivera-2020/stackexchange/eda/arresults.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Finally, we can create a recommender system for each user, using only the frequent-itemsets and association rules that we have created by collecting all the subdomains visited by a user, and then gathering all those subdomains that are associated with it. The downside of this though, is that there is no immediate ranking on which subdomain to recommend to a certain user. We can only make this recommendation as a bulk recommendation to the user.

Given below is a dataframe of recommendations, tagged in the column named prediction fo each unique user.

In [59]:
df_car.limit(5).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------------------+--------------------+
|UserId|               trans|          prediction|
+------+--------------------+--------------------+
| 714.0|[fitness, network...|[unix, dba, stats...|
|1033.0|[mathematica, hea...|[tex, codereview,...|
|1575.0|[webapps, mechani...|[tex, codereview,...|
|2077.0|[ai, networkengin...|[codereview, dba,...|
|2098.0|[parenting, mathe...|[tex, travel, sal...|
+------+--------------------+--------------------+

In [60]:
# Save car to csv
# df_car.toPandas().to_csv('s3://bdcc-rrivera-2020/stackexchange/eda/carresults.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Let's take a look at the recommendations for user 714 based on the frequent-itemsets and association rules that we have created.

In [61]:
df_car.limit(5).toPandas().iloc[0, 2]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['unix', 'dba', 'stats', 'english', 'softwareengineering', 'wordpress', 'electronics', 'physics', 'mathematica', 'blender', 'sharepoint', 'gamedev', 'salesforce', 'diy', 'gaming', 'scifi', 'workplace', 'ux', 'webmasters', 'money', 'travel', 'raspberrypi', 'cs', 'academia', 'graphicdesign', 'ell', 'codegolf', 'puzzling', 'chemistry', 'webapps', 'photo', 'arduino', 'mechanics', 'bitcoin', 'biology']

Maybe our user can hand-pick some of the interesting subdomains that he/she may see.

# Conclusion and Recommendation

For this project, we were able to wrangle Big Data and create two (2) implementations of recommender systems for StackExchange.
The first is through the use of a user-based collaborative filtering algorithm and the second is by using frequent-itemsets and association rules.

The first approach is more promising since it takes into account explicit ratings from each user. This allows us to recommend subdomains that a similar user would like. Thus, this approach is focused more towards the user. The second approach can be thought of as more owner-focused since it identifies the commonly visited subdomains and its associations. This could help the owners of StackExchange identify strong collaborations between subdomains, to encourage interaction and community engagement.

Some of the team's insights were:
- The big volume of user interaction with the website helped uncover subdomains frequently visited together; and
- A user-specific recommendation depends on browsing patterns of other users

With our findings, the team would like to recommend to the owners of StackExchange to:
- Place related subdomain links in the recommendation panes or conspicuous parts of the website; and
- Incentivize user engagement deeper into or across more subdomains

This approach or methodology is not only applicable to StackExchange, this could also be used to various platforms such as e-commerce platforms, social media platforms and literally any platform that has a rich database of user-item transactions.

# References and Acknowledgment

[1] Pdfs.semanticscholar.org. (2019). Available at: https://pdfs.semanticscholar.org/fb3a/fca3dcdc1d9d786fd56c3132f327caff111e.pdf. <br>
[2] Stack Abuse. (2019). Association Rule Mining via Apriori Algorithm in Python. Available at: https://stackabuse.com/association-rule-mining-via-apriori-algorithm-in-python/. <br>
[3] Medium. (2019). Building and Testing Recommender Systems With Surprise, Step-By-Step. Available at: https://towardsdatascience.com/building-and-testing-recommender-systems-with-surprise-step-by-step-d4ba702ef80b.

We would like to acknowledge Prof. Christian Alis for his lessons and guidance during the whole course of Big Data and Cloud Computing (BDCC).